In [1]:
import numpy as np
import datetime
from tqdm.notebook import tqdm

def choices(a_huge_key_list):
        L = len(a_huge_key_list)
        i = np.random.randint(0, L)
        return a_huge_key_list[i]

def get_adjacent_nodes_fast(array_links, set_links, link, U_labeled, U_labeled_first_value_array, U_labeled_second_value_array):
    
    stage = link[1]
    label = link[0]
    
    k, l = np.where(L_array == stage)

    inter_result = set().union(*[set_links & {(stage, j), (j, stage)} for j in np.unique(np.array([list(set_links)[i] for i in k]).flatten()).tolist() if stage != j])
    
    li_collect = list(inter_result)

    di_map1 = {a[0]: 'up' for a in li_collect if a[1] == stage}
    di_map2 = {a[1]: 'down' for a in li_collect if a[0] == stage}
    
    di_mapping = {**di_map1, **di_map2}
    
    candidates = [k for k in di_mapping.keys() if k != stage]
    candidate_labels = [x for x in U_labeled_first_value_array[np.isin(U_labeled_second_value_array, candidates)].tolist() if x < label]
    
    result_stages = [U_labeled[i-1][1] for i in candidate_labels]
    
    return {k: v for k,v in di_mapping.items() if k in result_stages}

def get_next_nodes(set_links, stages, L_array):
        
    for j in stages:
        
        if stages.size == 1:
            return stages[0]

        list_all_stages = L_array[np.where((L_array[:, 0] == j) | (L_array[:, 1] == j))].flatten()
        index1 = np.where(list_all_stages == j)
        list_all_connected_stages = np.delete(list_all_stages, index1)
        list_connected_unvisited_stages = list_all_connected_stages[np.isin(list_all_connected_stages, stages)]
        
        if list_connected_unvisited_stages.size == 1:
            return j
        else:
            return j * -1 # element occurs many times and to be processed later

def label_stages(I, L):

    I = I
    L = L

    N = list(range(1,I+1))
    U = list()
    l = 1
    
    while len(N) > 0:
        for ele in reversed(N):
            x = get_next_nodes(L, np.array(N), np.array(list(L)))
            if x > 0:
                U.append((l, x))
                N.remove(x)
                break

        l += 1
    
    return U

In [2]:
###### textbook example ######

I = 6
D = 6
my = 120 # units / month
sigma = 40
sl = 95
z = 1.64
h =  0.2
theta = 0

cost = np.array([3,4,1,2,3,1])
time = np.array([4,1,6,5,5,2])
cum_cost= np.array([3,4,8,2,13,14])
max_replenish = np.array([4,1,10,5,15,17])
std_devs = np.array([40,40,40,40,40,40])

D = [6]

L = {(1, 3), (2, 3), (3, 5), (4, 5), (5, 6)} 
U = [(1,1), (2,2), (3,4), (4,6), (5,5), (6,3)]

In [3]:
###### forward pass #######

L = set(L) # required for jumping across notebook cells

result_array = dict()
result_array_lamb = dict()
result_array_phi = dict()

final_lambdas = dict()
final_phis = dict()

U_labeled_first_value_array = np.array([x for x,y in U])
U_labeled_second_value_array = np.array([y for x,y in U])
L_first_value_array = np.array([x for x,y in L])
L_second_value_array = np.array([y for x,y in L])
L_array = np.array(list(L))

for idx, li in tqdm(enumerate(U), total=len(U), desc="Forward pass"):
    
    stage = li[1]
    it = stage - 1
    
    stream = get_adjacent_nodes_fast(L_array, L, li, U, U_labeled_first_value_array, U_labeled_second_value_array)
    
    x = max_replenish[it]-time[it]+1
    y = max_replenish[it]+1
    
    if stage in D:
        y = 1 # according to paper pg. 498
 
    array = np.zeros((x, y))
    
    for lam in range(x):
        for phi in range(y):
            array[lam, phi] = h*cum_cost[it]*z*std_devs[it]*np.sqrt(lam+time[it]-phi) # np.nan_to_num
            for t in stream.keys():
                if stream[t] == 'down':
                    array[lam, phi] += np.nanmin(result_array_lamb[t][phi:])
                elif stream[t] == 'up':
                    array[lam, phi] += np.nanmin(result_array_phi[t][:lam+1])
    
    if stage in D:
        lamb_cap = array.copy().flatten()
        phi_cap = np.nanmin(array, axis=1).flatten()
    else:
        lamb_cap = np.nanmin(array, axis=1).flatten()
        phi_cap = array.copy().flatten()
        
    phi_cap = np.zeros((1, y)).flatten()
    for phi in range(y):
        phi_cap[phi] = np.nanmin(array[max(0, phi-time[it]):, phi])
    
    result_array[stage] = array.copy()
    result_array_lamb[stage] = lamb_cap
    result_array_phi[stage] = phi_cap

    final_phis[stage] = result_array_phi[stage].argmin()
    final_lambdas[stage] = result_array_lamb[stage].argmin()

Forward pass:   0%|          | 0/6 [00:00<?, ?it/s]

c:\users\justi\miniconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in sqrt


In [4]:
###### backward pass #######

L = list(L) # required by algo

for l in tqdm(range(I-1, 0, -1), total=I-1, desc="Backward pass"):
        
    indices = np.where(U_labeled_first_value_array == l)[0]
    label, stage = [U[i] for i in indices][0]
        
    it = stage - 1
    
    indices = np.where(L_second_value_array == stage)[0].tolist() # list where stage is destination
    
    if len(indices) > 0:

        k, l = np.where(L_array == stage)
                
        # relevant origins in network
        origins = [L[i][0] for i in indices]
        
        ii1 = np.where(np.isin(U_labeled_second_value_array, origins))[0]
        origin_labels = [U[i][0] for i in ii1]
        correct_origins = [origins[i] for i in np.where(np.less(label, origin_labels) == True)[0]]
        
        for stage_to_be_processed in correct_origins:

            final_lambdas[stage] = final_phis[stage_to_be_processed]
            lambdaa = final_lambdas[stage]
            
            phii = np.nanargmin(result_array[stage][lambdaa, :])
            
            final_phis[stage] = phii
    else:
        
        indices = np.where(L_first_value_array == stage)[0].tolist() # list where stage is origin
        
        # relevant destinations in network
        destinations = [L[i][1] for i in indices]
        
        ii1 = np.where(np.isin(U_labeled_second_value_array, destinations))[0]
        destination_labels = [U[i][0] for i in ii1]
        correct_destinations = [destinations[i] for i in np.where(np.less(label, destination_labels) == True)[0]]
        
        for stage_to_be_processed in correct_destinations:

            if max_replenish[it] > final_lambdas[stage_to_be_processed]:
                final_phis[stage] = final_lambdas[stage_to_be_processed]
            
            lambdaa = np.nanargmin(result_array[stage][:, final_phis[stage]])
            
            final_lambdas[stage] = lambdaa

Backward pass:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
import pandas as pd

idx_list, lambda_list, phi_list, days_inv_list, ss_cost_list, ss_list = [], [], [], [], [], []

for a, b in zip(final_lambdas.items(), final_phis.items()):
    
    idx_list.append(a[0])
    lambda_list.append(int(a[1]))
    phi_list.append(b[1])
    days_inv_list.append(max(int(a[1]) + time[a[0]-1] - b[1], 0))
    ss_cost = h * cum_cost[a[0]-1] * z * std_devs[a[0]-1] * np.sqrt(max(int(a[1]) + time[a[0]-1] - b[1], 0))
    ss_cost_list.append(ss_cost)
    ss_list.append(ss_cost / (h * cum_cost[a[0]-1])) # formula from paper

di_results = {'Index': idx_list, 'Lambda': lambda_list, 'Phi': phi_list, 'Days_of_inventory': days_inv_list, 'Safety_stock_cost': ss_cost_list, 'Safety_stock': ss_list}

df_results = pd.DataFrame(di_results)
df_results.sort_values('Index')

,Index,Lambda,Phi,Days_of_inventory,Safety_stock_cost,Safety_stock
0,1,0,1,3,68.173520,113.622533
1,2,0,1,0,0.000000,0.000000
5,3,1,7,0,0.000000,0.000000
2,4,0,5,0,0.000000,0.000000
4,5,7,12,0,0.000000,0.000000
3,6,12,0,14,687.267629,245.452725
